## Function to download a product in streaming mode, showing a progress bar. 

In [ ]:
def downloadFile(url, headers, directory, total_length = 0) :
    """
    downloadFile(url, headers, directory)
    
    Download the file using streaming

    Parameters
    ----------
    url:
        The URL for the request

    headers:
        Headers (e.g. auth bearer token)

    directory:
        Directory for the download

    size:
        The size of the file

    """

    r = requests.get(url, headers=headers, stream=True)
    if r.status_code == 200:
        filename = path + get_filename_from_cd(r.headers.get('content-disposition'))
        print("Downloading " + filename)
        with open(filename, 'wb') as f:
            start = time.clock()
            print("File size is: %8.2f MB" % (total_length/(1024*1024)))
            dl = 0
            for chunk in r.iter_content(64738):
                dl += len(chunk)
                f.write(chunk)
                if total_length is not None: # no content length header
                    done = int(50 * dl / total_length)
                    #sys.stdout.write("\r[%s%s] %s kbps" % ('=' * done, ' ' * (50-done), dl//(time.clock() - start)))
                    print("\r[%s%s]  %8.2f Mbps" % ('=' * done, ' ' * (50-done), (dl//(time.clock() - start))/(1024*1024)), end='', flush=True)
                    #print ('')
                else:
                    if( dl % (1024)  == 0 ):
                        print("[%8.2f] MB downloaded, %8.2f kbps" % (dl / (1024 * 1024), (dl//(time.clock() - start))/1024))
            print("[%8.2f] MB downloaded, %8.2f kbps" % (dl / (1024 * 1024), (dl//(time.clock() - start))/1024))
            return (time.clock() - start)

## Function to get the filename from the HTTP response header - content-disposition (cd)

In [ ]:
def get_filename_from_cd(cd):
    """
    get_filename_from_cd(cd)

    Get the filename from content disposition
    Parameters
    ----------
    cd : content disposition
        From https://www.w3.org/Protocols/rfc2616/rfc2616-sec19.html
        The Content-Disposition response-header field has been proposed as a means for the origin server to suggest a default filename 
        if the user requests that the content is saved to a file. 
        This usage is derived from the definition of Content-Disposition in RFC 1806 [35].
    """  
    
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0][2:-1]

## Code snippet that relies on the above defined functions to download files. 
Note this is a code snippet and cannot be run stand-alone.

```python
counter = 0
for result in results['content']:
    externalUri = result['externalUri']
    product_size = result['fileSize']

    download_url = broker_address + '/datarequest/result/' + job_id + '?externalUri=' + urllib.parse.quote(externalUri)
    print( "Request: " + download_url)
    time_elapsed = downloadFile(download_url, headers, path, product_size)
    print( "Download complete...")
    print ("Time Elapsed: " + str(time_elapsed) + " seconds")

    counter+=1
    if counter > files_to_download:
        break
```        